# 十二、训练深度神经网络

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

### [Batch Normalization](https://zhuanlan.zhihu.com/p/38176412)

尽管使用 He 初始化和 ELU（或任何 ReLU 变体）可以显著减少训练开始阶段的梯度消失/爆炸问题，但不能保证在训练期间问题不会再次出现。

[在 2015 年的一篇论文中](https://links.jianshu.com/go?to=https%3A%2F%2Farxiv.org%2Fabs%2F1502.03167)，Sergey Ioffe 和 Christian Szegedy 提出了一种称为批归一化（Batch Normalization，BN）的方法来解决梯度消失/爆炸问题。该方法包括在每层的激活函数之前或之后在模型中添加操作。操作就是将输入平均值变为 0，方差变为 1，然后用两个新参数，一个做缩放，一个做偏移。换句话说，这个操作可以让模型学习到每层输入值的最佳缩放值和平均值。大大多数情况下，如果模型的第一层使用了 BN 层，则不用标准化训练集（比如使用`StandardScaler`）；BN 层做了标准化工作（虽然是近似的，每次每次只处理一个批次，但能做缩放和平移）。

为了对输入进行零居中（平均值是 0）和归一化，算法需要估计输入的均值和标准差。 它通过评估当前小批量输入的均值和标准差（因此命名为“批归一化”）来实现。 整个操作在公式中。


<div align=center><img width="500" height="300" src="./static/1.jpg"/></div>


#### 注意：

你可能会发现，训练相当缓慢，这是因为每个周期都因为使用 BN 而延长了时间。但是有了 BN，收敛的速度更快，需要的周期数更少。综合来看，需要的总时长变短了。

### 使用 Keras 实现批归一化

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10,activation='softmax')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 784)               3136      
_________________________________________________________________
dense_2 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_3 (Batch (None, 300)               1200      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1

可以看到每个 BN 层添加了四个参数：γ、 β、 μ 和 σ（例如，第一个 BN 层添加了 3136 个参数，即4 × 784）。后两个参数μ 和 σ是移动平均，不受反向传播影响，Keras 称其“不可训练”（如果将 BN 的总参数3,136 + 1,200 + 400除以 2，得到 2368，就是模型中总的不可训练的参数量）。

看下第一个 BN 层的参数。两个参数是可训练的（通过反向传播），两个不可训练：

In [7]:
[(var.name ,var.trainable) for var in model.layers[1].variables]

[('batch_normalization_2/gamma:0', True),
 ('batch_normalization_2/beta:0', True),
 ('batch_normalization_2/moving_mean:0', False),
 ('batch_normalization_2/moving_variance:0', False)]

当在 Keras 中创建一个 BN 层时，训练过程中，还会创建两个 Keras 在迭代时的操作。该操作会更新移动平均值。因为后端使用的是 TensorFlow，这些操作就是 TensorFlow 操作（第 12 章会讨论 TF 操作）：

In [12]:
model.layers[1].updates

D:\APYPLA~1\venv_dir\venv1\lib\site-packages\keras\engine\base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


[]

BN 的论文作者建议在激活函数之前使用 BN 层，而不是像前面的例子添加到后面。到底是前面还是后面好存在争议，取决于具体的任务 —— 你最好在数据集上试验一下哪种选择好。要在激活函数前添加 BN 层，必须将激活函数从隐藏层拿出来，单独做成一层。另外，因为 BN 层对每个输入有一个偏移参数，可以将前一层的偏置项去掉（设置use_bias=False）：

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization()
    keras.layers.Dense(300,kernel_initializer='he_normal',use_bias=False)
    keras.layers.BatchNormalization()
    keras.layers.Activation('elu'),
    keras.layers.Dense(100,kernel_initializer='he_normal',use_bias=False)
    keras.layers.BatchNormalization()
    keras.layers.Activation('elu'),
    keras.layers.Dense(10,activation='softmax')
])